In [34]:
import os, re, json
import commands
from glob import glob
import numpy as np
from prettytable import PrettyTable

In [2]:
!export PATH=$HOME/.local/bin:/cvmfs/cms-bril.cern.ch/brilconda/bin:$PATH

# Input data

In [3]:
loc = os.environ['HOME'] + '/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp'
loc += '/crab_ParkingBPH*_Run2018*_RDntuplizer_B2JpsiKst_191019'
print loc
loc = glob(loc)

/afs/cern.ch/user/o/ocerri/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp/crab_ParkingBPH*_Run2018*_RDntuplizer_B2JpsiKst_191019


In [33]:
lumiInfo = {}
for main_dir in loc:
    print main_dir.split('/')[-1]
    s = main_dir.split('/')[-1]
    idx = s.find('Run2018')
    era = s[idx+len('Run2018')]
    idx = s.find('ParkingBPH')
    part = int(s[idx+len('ParkingBPH')])
    if part != 1:
        continue
    lumiInfo[era] = {}

    cmd = 'brilcalc lumi -u /fb --precision 2f'
    cmd += ' --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json'
    cmd += ' --hltpath HLT_Mu\*_IP?_part0_v?'
    cmd += ' -i {}/processedLumis.json'.format(main_dir + '/results')

    status, output = commands.getstatusoutput(cmd)
    aux = output.split('#Summary:')[1]
    aux = aux.split('\n')
    for l in aux:
        if not l.startswith('| HLT'):
            continue
        l = l[1:-1]
        content = l.split(' | ')
        trgPath = content[0].replace(' ', '')
        lumi = float(content[-1].replace(' ', ''))
        if lumi < 0.02:
            continue
        lumiInfo[era][trgPath] = lumi
        print trgPath, lumi

crab_ParkingBPH1_Run2018D-05May2019promptD-v1_RDntuplizer_B2JpsiKst_191019
HLT_Mu12_IP6_part0_v2 5.25
HLT_Mu7_IP4_part0_v2 1.27
HLT_Mu8_IP3_part0_v3 0.08
HLT_Mu8_IP5_part0_v2 1.65
HLT_Mu8_IP6_part0_v2 1.65
HLT_Mu9_IP5_part0_v2 3.33
HLT_Mu9_IP6_part0_v3 4.12
crab_ParkingBPH2_Run2018D-05May2019promptD-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH3_Run2018D-05May2019promptD-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH1_Run2018A-05May2019-v1_RDntuplizer_B2JpsiKst_191019
HLT_Mu8_IP3_part0_v1 0.04
HLT_Mu8_IP3_part0_v2 0.03
HLT_Mu9_IP6_part0_v1 0.29
HLT_Mu9_IP6_part0_v2 0.48
crab_ParkingBPH2_Run2018A-05May2019-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH3_Run2018A-05May2019-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH4_Run2018A-05May2019-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH5_Run2018A-05May2019-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH6_Run2018A-05May2019-v1_RDntuplizer_B2JpsiKst_191019
crab_ParkingBPH1_Run2018B-05May2019-v2_RDntuplizer_B2JpsiKst_191019
HLT_Mu12_IP6

In [42]:
eras = list(np.sort(lumiInfo.keys()))
table = PrettyTable()

table.field_names = ['HLT path'] + eras

printed_paths = []
for e in eras:
    for t in lumiInfo[e]:
        if t in printed_paths:
            continue
        printed_paths.append(t)
        row = [t]
        for e in eras:
            if t in lumiInfo[e].keys():
                row.append(str(lumiInfo[e][t]))
            else:
                row.append(' - ')
        table.add_row(row)
print table

+-----------------------+------+------+------+------+
|        HLT path       |  A   |  B   |  C   |  D   |
+-----------------------+------+------+------+------+
|  HLT_Mu9_IP6_part0_v1 | 0.29 |  -   |  -   |  -   |
|  HLT_Mu9_IP6_part0_v2 | 0.48 | 0.38 |  -   |  -   |
|  HLT_Mu8_IP3_part0_v1 | 0.04 |  -   |  -   |  -   |
|  HLT_Mu8_IP3_part0_v2 | 0.03 | 0.06 |  -   |  -   |
|  HLT_Mu7_IP4_part0_v2 |  -   | 0.07 | 0.03 | 1.27 |
| HLT_Mu12_IP6_part0_v2 |  -   | 0.53 | 1.1  | 5.25 |
|  HLT_Mu8_IP3_part0_v3 |  -   | 0.07 |  -   | 0.08 |
|  HLT_Mu9_IP6_part0_v3 |  -   | 0.41 | 0.77 | 4.12 |
|  HLT_Mu9_IP5_part0_v2 |  -   | 0.27 | 0.54 | 3.33 |
|  HLT_Mu8_IP6_part0_v2 |  -   |  -   |  -   | 1.65 |
|  HLT_Mu8_IP5_part0_v2 |  -   |  -   |  -   | 1.65 |
+-----------------------+------+------+------+------+
